# Проект Определение стоимости автомобилей

## Введение


Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля. 
Постройте модель, которая умеет её определять. В вашем распоряжении данные о технических характеристиках, комплектации и ценах других автомобилей.
Критерии, которые важны заказчику:
- качество предсказания;
- время обучения модели;
- время предсказания модели.


## Описание данных

#### Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя
#### Целевой признак
- Price — цена (евро)

In [ ]:
!pip install -q catboost
!pip install -q phik

In [ ]:
!pip install 'lightgbm[scikit-learn]'

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, precision_score, make_scorer, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from phik import phik_matrix
from phik.report import plot_correlation_matrix
from sklearn.metrics import mean_squared_error
import warnings
from sklearn.svm import SVC, SVR
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from math import sqrt


RANDOM_STATE = 42
TEST_SIZE = 0.25

matplotlib_axes_logger.setLevel('ERROR')
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('max_colwidth', -1)

sns.set()

## Загрузка данных

In [ ]:
df_cars = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')
display(df_cars.head(5))
df_cars.info()

По загруженным данным можно сделать следующие выводы:
- размерность датасета 354369 х 16
- датасет содержит как числовые так и категориальные типы данных. Кроме полей которые содержат дату и имеют тип object (DateCrawled, DateCreated, LastSeen), все остальные типы соответствуют своим данным. Типы колонок DateCrawled, DateCreated, LastSeen надо перобразовать в соответствующие типы (date или timestamp)
- в некоторых колонках имеются пропуски

## Предобработка данных

- Колонки DateCrawled, DateCreated, LastSeen приведены к типу datetime64
- Остальные колонки типа object приведены к типу string

In [ ]:
df_cars_str_col_list = df_cars.select_dtypes(exclude=np.number).columns
df_cars[df_cars_str_col_list] = df_cars[df_cars_str_col_list].astype('string')

df_cars['DateCrawled'] = pd.to_datetime(df_cars['DateCrawled'])
df_cars['DateCreated'] = pd.to_datetime(df_cars['DateCreated'])
df_cars['LastSeen'] = pd.to_datetime(df_cars['LastSeen'])

df_cars.info()
df_cars.head(5)

Колонка NumberOfPictures содержит значения все равные 0. В связи с этим она признана неинформативной и удалена из датасета

In [ ]:
df_cars.describe()
df_cars = df_cars.drop(['NumberOfPictures'], axis = 1)
df_cars.info()

Для того, чтобы предварительно обнаружить дубликаты, надо определить набор аттрибутов на которых надо определять эти дубликаты. Во первых, надо из общего числа аттрибутов убрать целевой признак Price, потом убрать те входные признаки, которые не будут участвовать в работе модели.
На мой взгляд это такие служебные признаки как:
- DateCrawled — дата скачивания анкеты из базы
- DateCreated — дата создания анкеты
- LastSeen — дата последней активности пользователя

Обнаружено 34565 дубликатов для всех входных аттрибутов (исключая вышеперечисленные). Пока с ними ничего не буду делать, так как далее будет иметь место заполнение пропусков и я посмотрю на дубликаты после этой процедуры.

In [ ]:
list_for_duplicates = ["Price", "DateCrawled", "DateCreated", "LastSeen"]
print(f'Количество дубликатов: df_cars: {df_cars.duplicated(subset=df_cars.columns.difference(list_for_duplicates)).sum()}')


Ананлиз данных (метод info()) показал, что пропуски имеют место только в категориальных колонках (кроме колонки Brand) и в связи с этим принято решение заполнить их в пайплайне SimpleImputer модой для соответствующего аттрибута. После этого еще раз провести проверку на дубликаты и уже после этого удалить имеющиеся дубликаты.

In [ ]:
cat_col_list = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
def display_value_count(df, col_list):
    for c in col_list:
        print(f"------- {c} -------")
        display(df[c].value_counts(dropna = False))

display_value_count(df_cars, cat_col_list)

df_cars.info()

Использую пайплайн для применения SimpleImputer на этапе предобработки данных. В пайплайне использую SimpleImputer  для замены NaN на моду для колонок из списка cat_col_list с категориальными признаками так как пропуски имеют место только для этих колонок.


In [ ]:
df_cars = df_cars.replace({pd.NA: np.nan})
df_cars_dtypes = df_cars.dtypes

X = df_cars.drop(['Price'], axis=1)
y = df_cars['Price']

simple_imp_pipeline = Pipeline(
    [
        (
            'simpleImputer_nan', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
)

data_preprocessor = ColumnTransformer(
    [
        ('simple_imp_pipeline', simple_imp_pipeline, X.columns)
    ], 
    remainder='passthrough'
) 

pipe_final = Pipeline(
    [
        ('preprocessor', data_preprocessor)
    ]
)

df_cars_arr = pipe_final.fit_transform(X)
df_cars = pd.concat([pd.DataFrame(df_cars_arr, columns = X.columns),y], axis=1)
for col in df_cars.columns:
  df_cars[col] = df_cars[col].astype(df_cars_dtypes[col])
display_value_count(df_cars, cat_col_list)
df_cars.info()

После замены пропусков на моду еще раз проверил количество дубликатов и их число теперь увеличилось и равно 40351. Удаляю эти дубликаты. Если позже придется в рамках исправления возможных аномалий и выбросов заменять их на какие либо значения для определенных колонок - то процедуру проверки и удаления дубликатов на этом же множестве аттрибутов надо повторить.

In [ ]:
print(f'Количество дубликатов: df_cars: {df_cars.duplicated(subset=df_cars.columns.difference(list_for_duplicates)).sum()}')
df_cars = df_cars.drop_duplicates(subset=df_cars.columns.difference(list_for_duplicates))
print(f'Количество дубликатов: df_cars: {df_cars.duplicated(subset=df_cars.columns.difference(list_for_duplicates)).sum()}')

In [ ]:
df_cars.info()

## Исследовательский анализ данных

In [ ]:
def plot_sub_plot_hist_boxplot(df, col_list, title):
    for col in col_list:
        if col == 'id':
            continue
        fig, axes = plt.subplots(nrows=1, ncols=2)
        df[col].plot(kind="hist", title=col, figsize=(10, 5), ax = axes[0])
        sns.boxplot(df[col], ax = axes[1])
        plt.title(title)
        plt.show()

def plot_pie_plot_for_columns(df, col_list, title):
    for col in col_list:
        df[col].value_counts().plot(kind='pie', labels=None, label='', autopct='%1.0f%%', legend=True, figsize=(5,5), title=col);
        plt.title(f'{title} for {col}')
        plt.show()

def plot_bar_plot_for_columns(df, col_list, title, normalizer = 1):
    for col in col_list:
        print(col)
        ax = df[col].value_counts().plot(kind='bar',  label='',  figsize=(5,5), title=col);
        plt.title(f'{title} for {col}')
        if normalizer != 1:
          vals = ax.get_yticks()
          ax.set_yticklabels(['{:,.0%}'.format(x / normalizer) for x in vals])
        plt.show()

Для начала посмотрим на значения колонок с timestamps и RegistrationYear - как они сочетаются друг с другом

Из анализа графиков и вывода метода describe() для колонок *'DateCrawled', 'DateCreated', 'LastSeen'* видно, что данные по этим трем колонкам в основном сосредоточены в диапазоне 2016-01 - 2016-04. Выше правой границы значений нет, ниже левой границы есть небольшое количество выбросов до 2014-04

Проанализировав график и вывода метода decribe() атрибуту RegistrationYear - видно что его среднее и медиана находятся в районе 2003-2004 гг, но имеют место выбросы как в большую сторону вплоть до 9999, так и в меньшую сторону вплоть до 1000. Так как нет возможности выяснить как собирались эти данные и выяснить причину этих выбросов - то необходимо сначала избавиться от слишком явных выбросов - так как я не вижу тут больше других вариантов как и чем их можно было заменить. Так как max Значение аттрибута *DateCreated*  = 2016-04-07 то наибольшее валидное значение аттрибута RegistrationYear это 2016 г так как при создании анкеты нельзя указать автомобиль с годом регистрации в будущем (если это не ошибка ввода). Таким образом верхняя граница отсечки по аттрибуту RegistrationYear это 2016. Что касается нижней границы отсечки то возьмем ее условно равной 1950 годом - автомобили за более ранние даты от 1950 г до момента начала автомобильной эры (1900 г) представляют собой уже очень дорогие раритеты, ну а за период от 1000 до 1900 гг никаких автомобилей не было. В данном допущении я исхожу из того, что аттрибут RegistrationYear важен для работы модели, так как он на мой взгляд, является признаком, по которому можно охарактеризовать тот или иной автомобиль и мы не можем просто исключить его из входных признаков для модели.

In [ ]:
df_cars.loc[:,['DateCrawled', 'DateCreated', 'LastSeen']].plot.line();
display(df_cars.loc[:,['DateCrawled', 'DateCreated', 'LastSeen']].describe())
plot_sub_plot_hist_boxplot(df_cars, ['RegistrationYear'],'')
display(df_cars.loc[:,['RegistrationYear']].describe())

После отсечки гистограмма RegistrationYear приобрела боллее читаемый вид, графики для аттрибутов 'DateCrawled', 'DateCreated', 'LastSeen' особо не изменились. Далее нужно провести проверку на соответствие даты из аттрибутов DateCreated и RegistrationYear, при этом DateCreated приведенный к году должен быть больше или равен RegistrationYear, другими словами нельзя при создании анкеты указать RegistrationYear в будущем.

In [ ]:
df_cars_ryear_cut = df_cars[(df_cars['RegistrationYear'] <= 2016) & (df_cars['RegistrationYear'] >= 1950)]
df_cars_ryear_cut.info()
df_cars_ryear_cut.loc[:,['DateCrawled', 'DateCreated', 'LastSeen']].plot.line();
display(df_cars_ryear_cut.loc[:,['DateCrawled', 'DateCreated', 'LastSeen']].describe())

Тут я вижу, что записей для которых не выполняется указанное выше условие в датафрейме нет - то есть даты  DateCreated и RegistrationYear согласованы

In [ ]:
df_cars_ryear_cut['CreatedYear'] = df_cars_ryear_cut['DateCreated'].dt.strftime('%Y')
df_cars_ryear_cut['CreatedYear'] = df_cars_ryear_cut['CreatedYear'].astype('int64')
print(df_cars_ryear_cut['Brand'].count())
df_cars_ryear_cut_new = df_cars_ryear_cut[df_cars_ryear_cut['CreatedYear'] >= df_cars_ryear_cut['RegistrationYear']]
print(df_cars_ryear_cut_new['Brand'].count())

In [ ]:
df_cars_ryear_cut = df_cars_ryear_cut.drop(['CreatedYear'], axis=1)
df_cars_ryear_cut.info()

In [ ]:
plot_sub_plot_hist_boxplot(df_cars_ryear_cut, ['RegistrationYear'],'')
display(df_cars_ryear_cut.loc[:,['RegistrationYear']].describe())

Поменял тип для аттрибутов с object на string. PostalCode и RegistrationMonth также сделал категориальными переменными

In [ ]:
int_to_string_col_list = ['RegistrationMonth', 'PostalCode', 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'Repaired']
#df_cars.info()
df_cars_ryear_cut[int_to_string_col_list] = df_cars_ryear_cut[int_to_string_col_list].astype('string')
df_cars_ryear_cut.info()

Далее я вывожу статситику по категориальным признакам - 'VehicleType', 'Gearbox', 'Model', 'RegistrationMonth', 'FuelType', 'Brand', 'Repaired', 'PostalCode' - такие методы как describe, value_counts и bar plot (в bar plot не включил PostalCode так как там очень большое количество уникальных значений и график неинформативен и очень долго строится). Статистика по RegistrationMonth показывает что этот аттрибут принимает значения от 0 до 12 включительно то есть 13 различных значений - а месяцев в году 12. При этом частота повторения каждого месяца примерно одинакова и в связи с этим непонятно как быть с лишним месяцем - то ли брать за начальное значение 0 и тогда лишним будет месяц 12, то ли отсчет вести от месяца 1 и тогда лишним будет месяц 0 и также непонятно что делать с этим лишним месяцем - заменять его на моду множества - так частота примерно одинакова. Отбросить данные с "лишним" месяцем тоже нельзя - так как непонятно какой месяц считать лишним. Поскольку у меня нет информации с чем связана подобная ошибка ввода и как ее надо исправлять - я предлагаю оставить эту ситуаци как есть и ничего с ней не делать. В связи с чем в предсказании модели по аттрибуту Month будет заложена некая погрешность - можно в будущем попробовать удалить этот аттрибут и посмореть как это оразится на метрике.

In [ ]:
cat_list = ['VehicleType', 'Gearbox', 'Model', 'RegistrationMonth', 'FuelType', 'Brand', 'Repaired', 'PostalCode']
display(df_cars_ryear_cut.loc[:,cat_list].describe())
for c in cat_list:
    display(df_cars_ryear_cut[c].value_counts())

plot_bar_plot_for_columns(df_cars_ryear_cut, ['VehicleType', 'Gearbox', 'Model', 'RegistrationMonth', 'FuelType', 'Brand', 'Repaired'], "")

Анализ гистограмм числовых аттрибутов 'Power', 'Kilometer' показал, что:
- аттрибут Power варьируется в диапазоне от 0 до 20000 л.c. Величина 0 это очевидно ошибочное значение (и ряд значений от 0 до некоего минимального 50 л.c. также ошибочны), но и  значения свыше 1000 л.c. также являются ошибочными - 1000 л.c. это мощность гоночного болида. Поэтому я предлагаю сделать отсечку по этому аттрибуту по диапазону 50; 1000 так как я не знаю как вводились эти данные и с чем может связана такая ошибка
- аттрибут Kilometer принимает min и max значения равными 5000 и 150000 что кажется мне вполне валидными значениями для пробега

In [ ]:
num_list = ['Power', 'Kilometer']
plot_sub_plot_hist_boxplot(df_cars_ryear_cut, num_list, '')
display(df_cars_ryear_cut.loc[:,num_list].describe())

Графики для Power после обрезки

In [ ]:
df_cars_ryear_cut_power = df_cars_ryear_cut[(df_cars_ryear_cut['Power'] <= 1000) & (df_cars_ryear_cut['Power'] >= 50)]
plot_sub_plot_hist_boxplot(df_cars_ryear_cut_power, num_list, '')
display(df_cars_ryear_cut_power.loc[:,num_list].describe())

Scatter pair plot для числовых аттрибутов 'RegistrationYear', 'Kilometer', 'Power' не показал каких либо аномалий и выраженной линейной зависимости

In [ ]:
scatter_col_list = ['RegistrationYear', 'Kilometer', 'Power']
g = sns.pairplot(data = df_cars_ryear_cut_power, vars = scatter_col_list, height=3)
for ax in g.axes.flat:
    ax.tick_params("x", labelrotation=45)

Убираю из датафрейма df_cars_ryear_cut_power колонки 'DateCrawled', 'DateCreated', 'LastSeen' которые не нужны для моделирования так как представляют собой справочную информацию

In [ ]:
df_cars_ryear_cut_power = df_cars_ryear_cut_power.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1)
df_cars_ryear_cut_power.info()

Фик матрица была построена по всем аттрибутам датафрейма df_cars_ryear_cut_power за вычетом целевого аттрибута Price. Из анализа этой матрицы я вижу:
- корреляция между аттрибутами Model и Brand = 1. Это объяснимо так как Модель в данном случае является Брендом и наоборот - между ними взаимно однозначное соответствие (возможно отношение один ко многим для связи Brand -> Model)
- корреляция между Model и VehicleType довольно высока = 0.91
- корреляция между Brand и VehicleType ниже и = 0.62

В связи с этим предлагается убрать из входных аттрибутов Model так как он имеет высокую корреляцию с двуми другими аттрибутами Brand и VehicleType

In [ ]:
phik_overview = phik_matrix(df_cars_ryear_cut_power[df_cars_ryear_cut_power.columns.difference(['Price'])], interval_cols=scatter_col_list)

display(phik_overview)
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.2,
    figsize=(14, 8)
)

In [ ]:
df_cars_ryear_cut_power = df_cars_ryear_cut_power.drop(['Model'], axis=1)
df_cars_ryear_cut_power.info()

После удаления аттрибута Model фик матрица выглядит лучше.

In [ ]:
phik_overview = phik_matrix(df_cars_ryear_cut_power[df_cars_ryear_cut_power.columns.difference(['Price'])], interval_cols=scatter_col_list)

display(phik_overview)
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.2,
    figsize=(14, 8)
)

## Подготовка данных и обучение моделей

Выделяем датафреймы для входных и целевого признака, производим разбиение на тренировочную и тестовую выборку

In [ ]:
X = df_cars_ryear_cut_power.drop(['Price'], axis=1)
y = df_cars_ryear_cut_power['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)


Применяем модель CatBoostRegressor с параметрами iterations=10,learning_rate=1,depth=2 

In [ ]:
cat_columns = ['RegistrationMonth', 'VehicleType', 'Gearbox', 'FuelType', 'Brand', 'Repaired', 'PostalCode']
model = CatBoostRegressor(iterations=10,learning_rate=1,depth=2)
model.fit(X_train, y_train, cat_features=cat_columns)
preds_cbr = model.predict(X_test)

Готовим данные для LGBMRegressor (с заменой типа string на category) и строим модель LGBMRegressor

In [ ]:
X_lgbm_train = X_train.copy()
X_lgbm_test = X_test.copy()

def LGBMdfCreator(df):
    for c in df.columns:
        col_type = df[c].dtype
        if col_type == 'string':
            df[c] = df[c].astype('category')

LGBMdfCreator(X_lgbm_train)
LGBMdfCreator(X_lgbm_test)

X_lgbm_train.info()
X_lgbm_test.info()

gbm2 = LGBMRegressor(objective='rmse', random_state=RANDOM_STATE, early_stopping_rounds = 5, n_estimators=10000)
gbm2.fit(X_lgbm_train,  y_train, eval_set=[(X_lgbm_test, y_test)], eval_metric='rmse')
gbm2eval = gbm2.evals_result_

Далее формируем списки для энкодеров и скейлеров OneHotEncoder, OrdinalEncoder и MinMaxScaler и формируем ColumnTransformer для этих скейлеров для использования в пайплайне для non-boost модели DecisionTreeRegressor. Певоначальный прогон модели на основном датафрейме выдал ошибку **Unable to allocate 11.9 GiB for an array with shape (196129, 8113) and data type float64** что было связано с наличием категориального признака *PostalCode* c большим количеством категорий. Я считаю этот признак значимым для модели и две предыдущие boost модели успешно с ним справились и выдали метрику в рамках нужного диапазона. Но для успешной работы данной модели я дропну этот признак. Кроме того - в текущей версии sklearn = 0.24.1 нет метрики root_mean_squared_error и я ее реализовал через mean_squared_error (можно было проапгрейдить sklearn до максимальной версии но я не уверен что это не потянет за собой другие библиотеки и это надо делать централизовано в среде Практикума).

In [ ]:
X_train = X_train.drop(['PostalCode'], axis=1)
X_test = X_test.drop(['PostalCode'], axis=1)

In [ ]:

num_columns = list(X_train.select_dtypes(include=np.number))
ord_columns = ['RegistrationMonth']
#ohe_columns = ['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'Repaired', 'PostalCode']
ohe_columns = ['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'Repaired']
month_order_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

ohe_pipe = Pipeline(
    [
     ('ohe', OneHotEncoder(drop='first', handle_unknown='error', sparse=False))
    ]
    )

ord_pipe = Pipeline(
    [
     ('ohe', OrdinalEncoder(categories=[month_order_list]))
    ]
    )

data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ohe_columns),
     ('ord', ord_pipe, ord_columns),
     ('num', MinMaxScaler(), num_columns)
    ], 
    remainder='passthrough'
)

pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeRegressor(random_state=RANDOM_STATE))
])

param_grid = [
    {
        'models': [DecisionTreeRegressor(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 6),
        'models__max_features': range(2, 6)  
    },
]

mse = make_scorer(mean_squared_error,greater_is_better=False)
grid_search = GridSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring=mse,
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

result = pd.DataFrame(grid_search.cv_results_)
display(result[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score')) 


best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
 
print ('Лучшая модель и её параметры:\n', grid_search.best_estimator_)
print ('Метрика RMSE лучшей модели на тренировочной выборке:', sqrt(-grid_search.best_score_))

y_pred = best_model.predict(X_test)
rmse_test = sqrt(mean_squared_error(y_test, y_pred))
print ("Метрика RMSE лучшей модели на тестовой выборке:", rmse_test)

Вывод таблицы с результатами по трем моделям

In [ ]:
df_models = pd.DataFrame({'Model':['CatBoostRegressor', 'LGBMRegressor', 'DecisionTreeRegressor'], 'RMSE':['2330', '1799', '3492'], 'time':['5', '5', '33']}) 
display(df_models)

pred_gbm = gbm2.predict(X_lgbm_test)
rmse_test_catboost = sqrt(mean_squared_error(y_test, pred_gbm))
print ("Метрика RMSE лучшей модели LGBMRegressor на тестовой выборке:", rmse_test_catboost)

## Выводы

В рамках проекта Определение стоимости автомобилей были проведены следующие действия:
- загружены данные из источника данных *https://code.s3.yandex.net/datasets/autos.csv*
- проведен анализ данных на пропуски, найденные пропуски в категориальных колонках были заменены на моду по данному аттрибуту
- были определены колонки, которые не должны участвовать в работе модели и такие колонки были дропнуты
- проведен исследовательский анализ данных с проверкой на аномалии и выбросы, в соответствии с этим ряд данных был удален
- была проведена проверка на дубликаты только в рамках входных аттрибутов модели и найденный дубликаты были удалены 
- была построена фик матрица, на основани которой была удалена колонка, находящаяся в линейной зависимости с двумя другими колонками
- были построены три модели (данные по ним в таблице выше):
  - CatBoostRegressor
  - LGBMRegressor
  - DecisionTreeRegressor
- на основании полученных данных и условий сдачи проекта лучшей моделью балы признана модель LGBMRegressor с целевой метрикой RMSE = 1799 